## 141 期間内におけるmonth, weekのカバー率

In [1]:
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
from __future__ import print_function, division
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
import os
import sys
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import get_categorical_features, get_numeric_features
import datetime

from tqdm import tqdm
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import KFold
import warnings
import time
import sys
import datetime
from sklearn.metrics import mean_squared_error

os.listdir('../input/')
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_avtive_month']

### Data Load

In [2]:
def get_new_columns(name,aggs):
    return [name + '_' + k + '_' + agg for k in aggs.keys() for agg in aggs[k]]
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

df_train = utils.read_df_pkl('../input/train0*')
df_test = utils.read_df_pkl('../input/test0*')
df_hist = utils.read_df_pkl('../input/hist*0*')
df_new = utils.read_df_pkl('../input/new*0*')

df_train = reduce_mem_usage(df_train)
df_test  = reduce_mem_usage(df_test )
df_hist  = reduce_mem_usage(df_hist )
df_new   = reduce_mem_usage(df_new  )

train_test = pd.concat([df_train, df_test], axis=0)
df_trans = pd.concat([df_hist, df_new], axis=0)
train_test   = reduce_mem_usage(train_test  )
df_new   = reduce_mem_usage(df_new  )

100%|██████████| 3/3 [00:01<00:00,  2.60it/s]


Mem. usage decreased to  5.58 Mb (48.2% reduction)
Mem. usage decreased to  3.18 Mb (43.8% reduction)
Mem. usage decreased to 1971.22 Mb (40.8% reduction)
Mem. usage decreased to 140.41 Mb (37.5% reduction)
Mem. usage decreased to  9.00 Mb (0.0% reduction)
Mem. usage decreased to 140.41 Mb (0.0% reduction)


In [3]:
def get_new_columns(name,aggs):
    return [name + '_' + k + '_' + agg for k in aggs.keys() for agg in aggs[k]]

In [4]:
df = df_new
df = df[[key, 'authorized_flag', 'category_1', 'category_2', 'category_3', 'purchase_amount', 'installments', 'purchase_date']]
df.sort_values(by=[key, 'purchase_date'], inplace=True)
df['authorized_flag'] =  df['authorized_flag'].map(lambda x: 0 if x=='N' else 1)
df['purchase_date'] = pd.to_datetime(df['purchase_date'])
df['purchase_date_shift1'] =  df.groupby(key)['purchase_date'].shift(1)
df['diff_days'] =  (df['purchase_date'] - df['purchase_date_shift1']).dt.days
df_new = df.copy()
del df
gc.collect()

df = df_hist
df = df[[key, 'authorized_flag', 'category_1', 'category_2', 'category_3', 'purchase_amount', 'installments', 'purchase_date']]
df.sort_values(by=[key, 'purchase_date'], inplace=True)
df['authorized_flag'] =  df['authorized_flag'].map(lambda x: 0 if x=='N' else 1)
df['purchase_date'] = pd.to_datetime(df['purchase_date'])
df['purchase_date_shift1'] =  df['purchase_date'].shift(1)
df['diff_days'] =  (df['purchase_date_shift1'] - df['purchase_date']).dt.days
df_hist = df.copy()
del df
gc.collect()
df_hist.head()

,card_id,authorized_flag,category_1,category_2,category_3,purchase_amount,installments,purchase_date,purchase_date_shift1,diff_days
19095896,C_ID_00007093c1,1,N,3.0,C,0.037176,2,2017-02-14 14:00:43,NaT,NaN
19095775,C_ID_00007093c1,1,N,3.0,B,-0.467415,1,2017-02-14 15:47:45,2017-02-14 14:00:43,-1.0
19095845,C_ID_00007093c1,1,N,3.0,B,-0.572600,1,2017-02-16 15:37:58,2017-02-14 15:47:45,-2.0
19095866,C_ID_00007093c1,1,N,3.0,C,-0.331575,3,2017-02-20 12:19:01,2017-02-16 15:37:58,-4.0
19095808,C_ID_00007093c1,1,N,3.0,B,-0.483944,1,2017-03-03 00:24:15,2017-02-20 12:19:01,-11.0


In [7]:
auth1 = df_hist[df_hist['authorized_flag']==1]
auth0 = df_hist[df_hist['authorized_flag']==0]
df_train.set_index(key, inplace=True)
df_test.set_index(key, inplace=True)

### 連続week, month利用回数、期間内week, monthカバー率を特徴にする

In [8]:
# auth1
df = auth1.copy()
fname = 'auth1'
df['record'] = 1

aggs = {}
# aggs['authorized_flag'] = ['mean', 'std', 'sum']
aggs['purchase_amount'] = ['mean', 'max', 'min', 'sum']
aggs['diff_days'] = ['mean', 'std']
aggs['record'] = ['sum']

cat_list = ['category_1', 'category_2', 'category_3']
for cat in tqdm(cat_list):
    tmp = df.groupby([key, cat]).agg(aggs).reset_index()
    val_list = list(tmp[cat].value_counts().index)
    for val in tqdm(val_list):
        tmp2 = tmp[tmp[cat]==val]
        tmp2.set_index([key], inplace=True)
        use_cols = [col for col in tmp2.columns if not(col.count(cat))]
        tmp2 = tmp2[use_cols]
        new_columns = get_new_columns(f"{fname}_{cat}_{val}", aggs)
        tmp2.columns = new_columns
        df_train = df_train.join(tmp2)
        df_test = df_test.join(tmp2)
        
df_train.columns = [col.replace('.', '_') for col in df_train.columns]
df_test.columns = [col.replace('.', '_') for col in df_test.columns]
df_train.head()

100%|██████████| 3/3 [00:15<00:00,  5.54s/it]


,first_active_month,feature_1,feature_2,feature_3,target,auth1_category_1_N_purchase_amount_mean,auth1_category_1_N_purchase_amount_max,auth1_category_1_N_purchase_amount_min,auth1_category_1_N_purchase_amount_sum,auth1_category_1_N_diff_days_mean,auth1_category_1_N_diff_days_std,auth1_category_1_N_record_sum,auth1_category_1_Y_purchase_amount_mean,auth1_category_1_Y_purchase_amount_max,auth1_category_1_Y_purchase_amount_min,auth1_category_1_Y_purchase_amount_sum,auth1_category_1_Y_diff_days_mean,auth1_category_1_Y_diff_days_std,auth1_category_1_Y_record_sum,auth1_category_2_1_0_purchase_amount_mean,auth1_category_2_1_0_purchase_amount_max,auth1_category_2_1_0_purchase_amount_min,auth1_category_2_1_0_purchase_amount_sum,auth1_category_2_1_0_diff_days_mean,auth1_category_2_1_0_diff_days_std,auth1_category_2_1_0_record_sum,auth1_category_2_5_0_purchase_amount_mean,auth1_category_2_5_0_purchase_amount_max,auth1_category_2_5_0_purchase_amount_min,auth1_category_2_5_0_purchase_amount_sum,auth1_category_2_5_0_diff_days_mean,auth1_category_2_5_0_diff_days_std,auth1_category_2_5_0_record_sum,auth1_category_2_3_0_purchase_amount_mean,auth1_category_2_3_0_purchase_amount_max,auth1_category_2_3_0_purchase_amount_min,auth1_category_2_3_0_purchase_amount_sum,auth1_category_2_3_0_diff_days_mean,auth1_category_2_3_0_diff_days_std,auth1_category_2_3_0_record_sum,auth1_category_2_4_0_purchase_amount_mean,auth1_category_2_4_0_purchase_amount_max,auth1_category_2_4_0_purchase_amount_min,auth1_category_2_4_0_purchase_amount_sum,auth1_category_2_4_0_diff_days_mean,auth1_category_2_4_0_diff_days_std,auth1_category_2_4_0_record_sum,auth1_category_2_2_0_purchase_amount_mean,auth1_category_2_2_0_purchase_amount_max,auth1_category_2_2_0_purchase_amount_min,auth1_category_2_2_0_purchase_amount_sum,auth1_category_2_2_0_diff_days_mean,auth1_category_2_2_0_diff_days_std,auth1_category_2_2_0_record_sum,auth1_category_3_B_purchase_amount_mean,auth1_category_3_B_purchase_amount_max,auth1_category_3_B_purchase_amount_min,auth1_category_3_B_purchase_amount_sum,auth1_category_3_B_diff_days_mean,auth1_category_3_B_diff_days_std,auth1_category_3_B_record_sum,auth1_category_3_C_purchase_amount_mean,auth1_category_3_C_purchase_amount_max,auth1_category_3_C_purchase_amount_min,auth1_category_3_C_purchase_amount_sum,auth1_category_3_C_diff_days_mean,auth1_category_3_C_diff_days_std,auth1_category_3_C_record_sum,auth1_category_3_A_purchase_amount_mean,auth1_category_3_A_purchase_amount_max,auth1_category_3_A_purchase_amount_min,auth1_category_3_A_purchase_amount_sum,auth1_category_3_A_diff_days_mean,auth1_category_3_A_diff_days_std,auth1_category_3_A_record_sum
card_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
C_ID_92a2005557,2017-06,5,2,1,-0.820312,-0.637235,2.258394,-0.739395,-157.397018,-0.514170,15.587897,247.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.637350,2.258394,-0.739395,-155.513351,-0.495902,15.682156,244.0,-0.627888,-0.551864,-0.679559,-1.883663,-2.00000,1.732051,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.637235,2.258394,-0.739395,-157.397018,-0.514170,15.587897,247.0
C_ID_3d0044924f,2017-01,4,1,0,0.392822,-0.637977,0.873296,-0.740897,-197.772873,-0.658065,18.338930,310.0,-0.383123,4.630300,-0.742400,-11.110573,-2.000000,1.253566,29.0,-0.637977,0.873296,-0.740897,-197.772873,-0.658065,18.338930,310.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.700539,-0.439465,-0.742400,-190.546631,-0.481618,19.564437,272.0,-0.264935,4.630300,-0.682624,-17.220774,-1.953846,1.604980,65.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
C_ID_d639edf6cd,2016-08,2,2,0,0.687988,-0.678742,-0.145847,-0.730138,-27.828424,-0.609756,66.400255,41.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.713650,-0.705405,-0.719860,-2.854601,93.000000,208.134572,4.0,-0.674968,-0.145847,-0.730138,-24.973822,-10.72973,14.490703,37.0,NaN,Na

In [9]:
# auth0
df = auth0.copy()
fname = 'auth0'
df['record'] = 1

aggs = {}
# aggs['authorized_flag'] = ['mean', 'std', 'sum']
aggs['purchase_amount'] = ['mean', 'max', 'min', 'sum']
aggs['diff_days'] = ['mean', 'std']
aggs['record'] = ['sum']

cat_list = ['category_1', 'category_2', 'category_3']
for cat in tqdm(cat_list):
    tmp = df.groupby([key, cat]).agg(aggs).reset_index()
    val_list = list(tmp[cat].value_counts().index)
    for val in tqdm(val_list):
        tmp2 = tmp[tmp[cat]==val]
        tmp2.set_index([key], inplace=True)
        use_cols = [col for col in tmp2.columns if not(col.count(cat))]
        tmp2 = tmp2[use_cols]
        new_columns = get_new_columns(f"{fname}_{cat}_{val}", aggs)
        tmp2.columns = new_columns
        df_train = df_train.join(tmp2)
        df_test = df_test.join(tmp2)
        
df_train.columns = [col.replace('.', '_') for col in df_train.columns]
df_test.columns = [col.replace('.', '_') for col in df_test.columns]
df_train.head()

100%|██████████| 3/3 [00:04<00:00,  1.37s/it]


,first_active_month,feature_1,feature_2,feature_3,target,auth1_category_1_N_purchase_amount_mean,auth1_category_1_N_purchase_amount_max,auth1_category_1_N_purchase_amount_min,auth1_category_1_N_purchase_amount_sum,auth1_category_1_N_diff_days_mean,auth1_category_1_N_diff_days_std,auth1_category_1_N_record_sum,auth1_category_1_Y_purchase_amount_mean,auth1_category_1_Y_purchase_amount_max,auth1_category_1_Y_purchase_amount_min,auth1_category_1_Y_purchase_amount_sum,auth1_category_1_Y_diff_days_mean,auth1_category_1_Y_diff_days_std,auth1_category_1_Y_record_sum,auth1_category_2_1_0_purchase_amount_mean,auth1_category_2_1_0_purchase_amount_max,auth1_category_2_1_0_purchase_amount_min,auth1_category_2_1_0_purchase_amount_sum,auth1_category_2_1_0_diff_days_mean,auth1_category_2_1_0_diff_days_std,auth1_category_2_1_0_record_sum,auth1_category_2_5_0_purchase_amount_mean,auth1_category_2_5_0_purchase_amount_max,auth1_category_2_5_0_purchase_amount_min,auth1_category_2_5_0_purchase_amount_sum,auth1_category_2_5_0_diff_days_mean,auth1_category_2_5_0_diff_days_std,auth1_category_2_5_0_record_sum,auth1_category_2_3_0_purchase_amount_mean,auth1_category_2_3_0_purchase_amount_max,auth1_category_2_3_0_purchase_amount_min,auth1_category_2_3_0_purchase_amount_sum,auth1_category_2_3_0_diff_days_mean,auth1_category_2_3_0_diff_days_std,auth1_category_2_3_0_record_sum,auth1_category_2_4_0_purchase_amount_mean,auth1_category_2_4_0_purchase_amount_max,auth1_category_2_4_0_purchase_amount_min,auth1_category_2_4_0_purchase_amount_sum,auth1_category_2_4_0_diff_days_mean,auth1_category_2_4_0_diff_days_std,auth1_category_2_4_0_record_sum,auth1_category_2_2_0_purchase_amount_mean,auth1_category_2_2_0_purchase_amount_max,auth1_category_2_2_0_purchase_amount_min,auth1_category_2_2_0_purchase_amount_sum,auth1_category_2_2_0_diff_days_mean,auth1_category_2_2_0_diff_days_std,auth1_category_2_2_0_record_sum,auth1_category_3_B_purchase_amount_mean,auth1_category_3_B_purchase_amount_max,auth1_category_3_B_purchase_amount_min,auth1_category_3_B_purchase_amount_sum,auth1_category_3_B_diff_days_mean,auth1_category_3_B_diff_days_std,auth1_category_3_B_record_sum,auth1_category_3_C_purchase_amount_mean,auth1_category_3_C_purchase_amount_max,auth1_category_3_C_purchase_amount_min,auth1_category_3_C_purchase_amount_sum,auth1_category_3_C_diff_days_mean,auth1_category_3_C_diff_days_std,auth1_category_3_C_record_sum,auth1_category_3_A_purchase_amount_mean,auth1_category_3_A_purchase_amount_max,auth1_category_3_A_purchase_amount_min,auth1_category_3_A_purchase_amount_sum,auth1_category_3_A_diff_days_mean,auth1_category_3_A_diff_days_std,auth1_category_3_A_record_sum,auth0_category_1_N_purchase_amount_mean,auth0_category_1_N_purchase_amount_max,auth0_category_1_N_purchase_amount_min,auth0_category_1_N_purchase_amount_sum,auth0_category_1_N_diff_days_mean,auth0_category_1_N_diff_days_std,auth0_category_1_N_record_sum,auth0_category_1_Y_purchase_amount_mean,auth0_category_1_Y_purchase_amount_max,auth0_category_1_Y_purchase_amount_min,auth0_category_1_Y_purchase_amount_sum,auth0_category_1_Y_diff_days_mean,auth0_category_1_Y_diff_days_std,auth0_category_1_Y_record_sum,auth0_category_2_1_0_purchase_amount_mean,auth0_category_2_1_0_purchase_amount_max,auth0_category_2_1_0_purchase_amount_min,auth0_category_2_1_0_purchase_amount_sum,auth0_category_2_1_0_diff_days_mean,auth0_category_2_1_0_diff_days_std,auth0_category_2_1_0_record_sum,auth0_category_2_3_0_purchase_amount_mean,auth0_category_2_3_0_purchase_amount_max,auth0_category_2_3_0_purchase_amount_min,auth0_category_2_3_0_purchase_amount_sum,auth0_category_2_3_0_diff_days_mean,auth0_category_2_3_0_diff_days_std,auth0_category_2_3_0_record_sum,auth0_category_2_5_0_purchase_amount_mean,auth0_category_2_5_0_purchase_amount_max,auth0_category_2_5_0_purchase_amount_min,auth0_category_2_5_0_purchase_amount_sum,auth0_category_2_5_0_diff_days_mean,auth0_category_2_5_0_diff_days_std,auth0_category_2_5_0_record_sum,auth0_category_2_4_0_

In [10]:
# new
df = df_new.copy()
fname = 'new'
df['record'] = 1

aggs = {}
# aggs['authorized_flag'] = ['mean', 'std', 'sum']
aggs['purchase_amount'] = ['mean', 'max', 'min', 'sum']
aggs['diff_days'] = ['mean', 'std']
aggs['record'] = ['sum']

cat_list = ['category_1', 'category_2', 'category_3']
for cat in tqdm(cat_list):
    tmp = df.groupby([key, cat]).agg(aggs).reset_index()
    val_list = list(tmp[cat].value_counts().index)
    for val in tqdm(val_list):
        tmp2 = tmp[tmp[cat]==val]
        tmp2.set_index([key], inplace=True)
        use_cols = [col for col in tmp2.columns if not(col.count(cat))]
        tmp2 = tmp2[use_cols]
        new_columns = get_new_columns(f"{fname}_{cat}_{val}", aggs)
        tmp2.columns = new_columns
        df_train = df_train.join(tmp2)
        df_test = df_test.join(tmp2)
        
df_train.columns = [col.replace('.', '_') for col in df_train.columns]
df_test.columns = [col.replace('.', '_') for col in df_test.columns]
df_train.head()

100%|██████████| 3/3 [00:04<00:00,  1.31s/it]


,first_active_month,feature_1,feature_2,feature_3,target,auth1_category_1_N_purchase_amount_mean,auth1_category_1_N_purchase_amount_max,auth1_category_1_N_purchase_amount_min,auth1_category_1_N_purchase_amount_sum,auth1_category_1_N_diff_days_mean,auth1_category_1_N_diff_days_std,auth1_category_1_N_record_sum,auth1_category_1_Y_purchase_amount_mean,auth1_category_1_Y_purchase_amount_max,auth1_category_1_Y_purchase_amount_min,auth1_category_1_Y_purchase_amount_sum,auth1_category_1_Y_diff_days_mean,auth1_category_1_Y_diff_days_std,auth1_category_1_Y_record_sum,auth1_category_2_1_0_purchase_amount_mean,auth1_category_2_1_0_purchase_amount_max,auth1_category_2_1_0_purchase_amount_min,auth1_category_2_1_0_purchase_amount_sum,auth1_category_2_1_0_diff_days_mean,auth1_category_2_1_0_diff_days_std,auth1_category_2_1_0_record_sum,auth1_category_2_5_0_purchase_amount_mean,auth1_category_2_5_0_purchase_amount_max,auth1_category_2_5_0_purchase_amount_min,auth1_category_2_5_0_purchase_amount_sum,auth1_category_2_5_0_diff_days_mean,auth1_category_2_5_0_diff_days_std,auth1_category_2_5_0_record_sum,auth1_category_2_3_0_purchase_amount_mean,auth1_category_2_3_0_purchase_amount_max,auth1_category_2_3_0_purchase_amount_min,auth1_category_2_3_0_purchase_amount_sum,auth1_category_2_3_0_diff_days_mean,auth1_category_2_3_0_diff_days_std,auth1_category_2_3_0_record_sum,auth1_category_2_4_0_purchase_amount_mean,auth1_category_2_4_0_purchase_amount_max,auth1_category_2_4_0_purchase_amount_min,auth1_category_2_4_0_purchase_amount_sum,auth1_category_2_4_0_diff_days_mean,auth1_category_2_4_0_diff_days_std,auth1_category_2_4_0_record_sum,auth1_category_2_2_0_purchase_amount_mean,auth1_category_2_2_0_purchase_amount_max,auth1_category_2_2_0_purchase_amount_min,auth1_category_2_2_0_purchase_amount_sum,auth1_category_2_2_0_diff_days_mean,auth1_category_2_2_0_diff_days_std,auth1_category_2_2_0_record_sum,auth1_category_3_B_purchase_amount_mean,auth1_category_3_B_purchase_amount_max,auth1_category_3_B_purchase_amount_min,auth1_category_3_B_purchase_amount_sum,auth1_category_3_B_diff_days_mean,auth1_category_3_B_diff_days_std,auth1_category_3_B_record_sum,auth1_category_3_C_purchase_amount_mean,auth1_category_3_C_purchase_amount_max,auth1_category_3_C_purchase_amount_min,auth1_category_3_C_purchase_amount_sum,auth1_category_3_C_diff_days_mean,auth1_category_3_C_diff_days_std,auth1_category_3_C_record_sum,auth1_category_3_A_purchase_amount_mean,auth1_category_3_A_purchase_amount_max,auth1_category_3_A_purchase_amount_min,auth1_category_3_A_purchase_amount_sum,auth1_category_3_A_diff_days_mean,auth1_category_3_A_diff_days_std,auth1_category_3_A_record_sum,auth0_category_1_N_purchase_amount_mean,auth0_category_1_N_purchase_amount_max,auth0_category_1_N_purchase_amount_min,auth0_category_1_N_purchase_amount_sum,auth0_category_1_N_diff_days_mean,auth0_category_1_N_diff_days_std,auth0_category_1_N_record_sum,auth0_category_1_Y_purchase_amount_mean,auth0_category_1_Y_purchase_amount_max,auth0_category_1_Y_purchase_amount_min,auth0_category_1_Y_purchase_amount_sum,auth0_category_1_Y_diff_days_mean,auth0_category_1_Y_diff_days_std,auth0_category_1_Y_record_sum,auth0_category_2_1_0_purchase_amount_mean,auth0_category_2_1_0_purchase_amount_max,auth0_category_2_1_0_purchase_amount_min,auth0_category_2_1_0_purchase_amount_sum,auth0_category_2_1_0_diff_days_mean,auth0_category_2_1_0_diff_days_std,auth0_category_2_1_0_record_sum,auth0_category_2_3_0_purchase_amount_mean,auth0_category_2_3_0_purchase_amount_max,auth0_category_2_3_0_purchase_amount_min,auth0_category_2_3_0_purchase_amount_sum,...,auth0_category_3_B_diff_days_std,auth0_category_3_B_record_sum,auth0_category_3_A_purchase_amount_mean,auth0_category_3_A_purchase_amount_max,auth0_category_3_A_purchase_amount_min,auth0_category_3_A_purchase_amount_sum,auth0_category_3_A_diff_days_mean,auth0_category_3_A_diff_days_std,auth0_category_3_A_record_sum,auth0_category_3_C_purchase_amount_mean,auth0_category_3_C_purchase_amo

In [12]:
df_train.shape

(201917, 229)

### make ratio feature

In [11]:
base_train = df_train.reset_index()[key].to_frame()
base_test = df_test.reset_index()[key].to_frame()

feat_name_list = ['amount_mean', 'amount_max', 'amount_min', 'amount_sum', 'days_mean', 'days_std']
for feat_name in feat_name_list:

    feat_col = [col for col in df_train.columns if col.count(feat_name)]
    cat1_col = [col for col in feat_col if col.count('category_1')]
    cat2_col = [col for col in feat_col if col.count('category_2')]
    cat3_col = [col for col in feat_col if col.count('category_3')]
    df = pd.concat([df_train, df_test], axis=0)
    tmp = df[feat_col].fillna(0)
    
    type_list = ['auth1', 'auth0', 'new']
    for cols in tqdm([cat1_col, cat2_col, cat3_col]):
        df_cat = tmp[cols]
        for data in tqdm(type_list):
            use_cols = [col for col in cols if col.count(data)]
            tmp2 = df_cat[use_cols]
            tmp2['sum'] = tmp2.sum(axis=1)
            for col in use_cols:
                tmp2[f"{col}_ratio"] = tmp2[col].values / tmp2['sum'].values
            ratio_cols = [col for col in tmp2.columns if col.count('ratio')]
            tmp2 = tmp2[ratio_cols]
            base_train = base_train.merge(tmp2.reset_index(), how='inner', on=key)
            base_test = base_test.merge(tmp2.reset_index(), how='inner', on=key)
            
base_train.head()

100%|██████████| 3/3 [00:07<00:00,  2.41s/it]


,card_id,auth1_category_1_N_purchase_amount_mean_ratio,auth1_category_1_Y_purchase_amount_mean_ratio,auth0_category_1_N_purchase_amount_mean_ratio,auth0_category_1_Y_purchase_amount_mean_ratio,new_category_1_N_purchase_amount_mean_ratio,new_category_1_Y_purchase_amount_mean_ratio,auth1_category_2_1_0_purchase_amount_mean_ratio,auth1_category_2_5_0_purchase_amount_mean_ratio,auth1_category_2_3_0_purchase_amount_mean_ratio,auth1_category_2_4_0_purchase_amount_mean_ratio,auth1_category_2_2_0_purchase_amount_mean_ratio,auth0_category_2_1_0_purchase_amount_mean_ratio,auth0_category_2_3_0_purchase_amount_mean_ratio,auth0_category_2_5_0_purchase_amount_mean_ratio,auth0_category_2_4_0_purchase_amount_mean_ratio,auth0_category_2_2_0_purchase_amount_mean_ratio,new_category_2_1_0_purchase_amount_mean_ratio,new_category_2_NA_purchase_amount_mean_ratio,new_category_2_3_0_purchase_amount_mean_ratio,new_category_2_5_0_purchase_amount_mean_ratio,new_category_2_4_0_purchase_amount_mean_ratio,new_category_2_2_0_purchase_amount_mean_ratio,auth1_category_3_B_purchase_amount_mean_ratio,auth1_category_3_C_purchase_amount_mean_ratio,auth1_category_3_A_purchase_amount_mean_ratio,auth0_category_3_B_purchase_amount_mean_ratio,auth0_category_3_A_purchase_amount_mean_ratio,auth0_category_3_C_purchase_amount_mean_ratio,new_category_3_A_purchase_amount_mean_ratio,new_category_3_B_purchase_amount_mean_ratio,new_category_3_C_purchase_amount_mean_ratio,new_category_3_NA_purchase_amount_mean_ratio,auth1_category_1_N_purchase_amount_max_ratio,auth1_category_1_Y_purchase_amount_max_ratio,auth0_category_1_N_purchase_amount_max_ratio,auth0_category_1_Y_purchase_amount_max_ratio,new_category_1_N_purchase_amount_max_ratio,new_category_1_Y_purchase_amount_max_ratio,auth1_category_2_1_0_purchase_amount_max_ratio,auth1_category_2_5_0_purchase_amount_max_ratio,auth1_category_2_3_0_purchase_amount_max_ratio,auth1_category_2_4_0_purchase_amount_max_ratio,auth1_category_2_2_0_purchase_amount_max_ratio,auth0_category_2_1_0_purchase_amount_max_ratio,auth0_category_2_3_0_purchase_amount_max_ratio,auth0_category_2_5_0_purchase_amount_max_ratio,auth0_category_2_4_0_purchase_amount_max_ratio,auth0_category_2_2_0_purchase_amount_max_ratio,new_category_2_1_0_purchase_amount_max_ratio,new_category_2_NA_purchase_amount_max_ratio,new_category_2_3_0_purchase_amount_max_ratio,new_category_2_5_0_purchase_amount_max_ratio,new_category_2_4_0_purchase_amount_max_ratio,new_category_2_2_0_purchase_amount_max_ratio,auth1_category_3_B_purchase_amount_max_ratio,auth1_category_3_C_purchase_amount_max_ratio,auth1_category_3_A_purchase_amount_max_ratio,auth0_category_3_B_purchase_amount_max_ratio,auth0_category_3_A_purchase_amount_max_ratio,auth0_category_3_C_purchase_amount_max_ratio,new_category_3_A_purchase_amount_max_ratio,new_category_3_B_purchase_amount_max_ratio,new_category_3_C_purchase_amount_max_ratio,new_category_3_NA_purchase_amount_max_ratio,auth1_category_1_N_purchase_amount_min_ratio,auth1_category_1_Y_purchase_amount_min_ratio,auth0_category_1_N_purchase_amount_min_ratio,auth0_category_1_Y_purchase_amount_min_ratio,new_category_1_N_purchase_amount_min_ratio,new_category_1_Y_purchase_amount_min_ratio,auth1_category_2_1_0_purchase_amount_min_ratio,auth1_category_2_5_0_purchase_amount_min_ratio,auth1_category_2_3_0_purchase_amount_min_ratio,auth1_category_2_4_0_purchase_amount_min_ratio,auth1_category_2_2_0_purchase_amount_min_ratio,auth0_category_2_1_0_purchase_amount_min_ratio,auth0_category_2_3_0_purchase_amount_min_ratio,auth0_category_2_5_0_purchase_amount_min_ratio,auth0_category_2_4_0_purchase_amount_min_ratio,auth0_category_2_2_0_purchase_amount_min_ratio,new_category_2_1_0_purchase_amount_min_ratio,new_category_2_NA_purchase_amount_min_ratio,new_category_2_3_0_purchase_amount_min_ratio,new_category_2_5_0_purchase_amount_min_ratio,new_category_2_4_0_purchase_amount_min_ratio,new_category_2_2_0_purchase_amount_min_ratio,auth1_category_3_B_purchase_amount_min_ratio,auth1_categor

In [15]:
fname = '146_cat'
fname = '155_ffm'
ignore_features = ['first_active_month', 'card_id', target]
df = base_train
df = base_test

for col in df_train.columns:
    if col in ignore_features: continue
    if col.count('feature_'):continue
#     if not(col.count('ratio')):continue
    utils.to_pkl_gzip(path = f'../features/1_first_valid/{fname}_train_{col}@', obj=df[col].values)
    utils.to_pkl_gzip(path = f'../features/1_first_valid/{fname}_test_{col}@', obj=df[col].values)